# Composition and inheritance

La herencia es una de las herramientas más poderosas y utilizadas, pero tiene algunos problemas, el principal de ellos es que cada vez que heredamos de una clase padre, estamos creando una nueva que está estrechamente acoplado con su padre, y como sabemos, el acoplamiento es una de las cosas que se desean reducir tanto como se pueda cuando se diseña software.

Normalmente se relaciona la herencia con poder reutilizar código ya creado, y esto no está bien, a pesar de que se debe reutilizar el código tanto como se pueda, no tenemos que utilizar la herencia para ello, la forma correcta de hacer esto es tener objetos altamente cohesivos que puedan componerse fácilmente y que funcionen en múltiples contexto.

### Cuando utilizar la herencia?

Tenemos que tener cuidado al crear una clase heredada, por un lado, tiene la ventaja de que obtenemos todo el código de los métodos de la clase principal de forma gratuita, pero por otro lado , los estamos llevando a todos a una nueva clase, lo que significa que podríamos estar poniendo demasiada funcionalidad en una nueva definición.

Al crear una nueva subclase, tenemos que pensar si realmente va a usar todos los métodos que acaba de heredar, como una heurística para ver si la clase está definida correctamente. Si, en cambio, descubrimos que no necesitamos la mayoría de los métodos y tenemos que anularlos o reemplazarlos, este es un error de diseño que podría deberse a varias razones

La superclase está vagamente definida y contiene demasiada responsabilidad, en lugar de una interfaz bien definida. 

La subclase no es una especialización adecuada de la superclase que está tratando de extender. Un buen caso para usar la herencia es el tipo de situación cuando tiene una clase que define ciertos componentes con su comportamiento que están definidos por la interfaz de esta clase (sus métodos y atributos públicos), y luego necesita especializar esta clase para crear objetos que hagan lo mismo pero con algo más agregado, o con algo en particular partes de su comportamiento cambiaron.

Otro caso de buen uso de la herencia es cuando nuestra clase no tiene definido ninguno de los métodos, es decir una interfaz que no tiene implementado el comporamiento de los métodos y son las clases heredadas las que definen este comporamiento.


### Herencia múltiple en python

(El problema del diamante) Cuando una clase se extiende desde dos o más, y todas esas clases también se extienden desde otras clases base, el los de abajo tendrán múltiples formas de resolver los métodos que provienen de las clases de nivel superior. La pregunta es, ¿cuál de estas implementaciones se usa?

In [4]:
class BaseModule:
    module_name = "top"
    
    def __init__(self, module_name):
        self.name = module_name

    def __str__(self):
        return f"{self.module_name}:{self.name}"

class BaseModule1(BaseModule):
    module_name = "module-1"

class BaseModule2(BaseModule):
    module_name = "module-2"

class BaseModule3(BaseModule):
    module_name = "module-3"

class ConcreteModuleA12(BaseModule1, BaseModule2):
    """Extend 1 & 2"""
    
class ConcreteModuleB23(BaseModule2, BaseModule3):
    """Extend 2 & 3"""
    
# habrá colisiones en python?

str(ConcreteModuleA12("test"))

'module-1:test'

No hay colisión, python resuelve esto usando un algoritmo llamado Linearización C3 o MRO, el cual define una forma deterministica en la cual los métodos vana  ser llamada, y de hecho, podemos pedirle específicamente a la clase su orden de resolución: así:

In [5]:
[cls.__name__ for cls in ConcreteModuleA12.mro()]

['ConcreteModuleA12', 'BaseModule1', 'BaseModule2', 'BaseModule', 'object']

### Mixins

Un mixin es una clase base que encapsula algunos comportamientos comunes con el objetivo de reutilizar el código.

Por lo general, una clase mixin no es útil por sí sola, y extender esta clase solo ciertamente no funcionará, porque la mayoría de las veces depende de métodos y propiedades que se definen en otras clases.

La idea es usar clases mixin junto con otras, a través de herencia múltiple, de modo que los métodos o propiedades utilizados en el mixin estarán disponibles.Por ej.

In [7]:
class BaseTokenizer:
    
    def __init__(self, str_token):
        self.str_token = str_token
    
    def __iter__(self):
        yield from self.str_token.split("-")

In [8]:
tk = BaseTokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
list(tk)

['28a2320b', 'fd3f', '4627', '9792', 'a2b38e3c46b0']

Pero ahora queremos que los valores se envíen en mayúsculas, sin alterar la clase base. Para este simple ejemplo, podríamos crear una nueva clase, pero imaginemos que muchas clases ya se están extendiendo desde BaseTokenizer, y no queremos reemplazarlas todas. Podemos mezclar una nueva clase en la jerarquía que maneja esta transformación:

In [9]:
class UpperIterableMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())

class Tokenizer(UpperIterableMixin, BaseTokenizer):
    pass

No necesita ningún código porque aprovecha el mixin. Este tipo de mezcla actúa como una especie de decorador. Según lo que acabamos de ver, Tokenizer tomará __iter__ del mixin, y este, a su vez, delega a la siguiente clase en la línea (llamando a super ()), que es BaseTokenizer, pero
convierte sus valores a mayúsculas, creando el efecto deseado.

In [12]:
z = Tokenizer("a-b-c")
list(z)

['A', 'B', 'C']